# **Random Forest**
- Using data that was annotated
- Training it on the unbalanced data
- Using avgpool
- 1 second window
- Entropy is the measure of information contained in a state


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_1s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[-2.10519694e-02, -2.64618150e-03, -1.09052584e-02, ...,
          -1.14497980e-02,  0.00000000e+00, -1.33858034e-02],
         [-1.99002311e-01, -2.20890388e-01, -2.76137829e-01, ...,
          -2.59782881e-01, -2.09464088e-01, -1.87755451e-01],
         [-2.39491284e-01, -2.51040578e-01, -2.89648235e-01, ...,
          -2.82749534e-01, -2.52160460e-01, -2.39700243e-01],
         ...,
         [-9.98796940e-01, -9.98772740e-01, -9.98712122e-01, ...,
          -9.96163070e-01, -9.97696698e-01, -9.98156428e-01],
         [-9.99429047e-01, -9.99434769e-01, -9.99466956e-01, ...,
          -9.99509513e-01, -9.99450743e-01, -9.99419093e-01],
         [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
          -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
 
        [[-1.80637967e-02, -3.36689858e-08, -7.33598135e-03, ...,
          -1.05369762e-02, -8.22041358e-04, -1.49706351e-02],
         [-1.93169340e-01, -2.11562306e-01, -2.58488506e-01, ...,
   

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 128, 87)

array([[-1.73581708e-02,  0.00000000e+00, -8.14180914e-03, ...,
        -9.85445548e-03, -1.71658248e-04, -1.32468548e-02],
       [-1.93985939e-01, -2.12422282e-01, -2.69561797e-01, ...,
        -2.80415505e-01, -2.17691883e-01, -1.94388315e-01],
       [-2.68526703e-01, -2.86887318e-01, -3.42721850e-01, ...,
        -3.53653818e-01, -2.91956156e-01, -2.68852562e-01],
       ...,
       [-9.99117315e-01, -9.99106705e-01, -9.99057472e-01, ...,
        -9.98603582e-01, -9.98687506e-01, -9.98692393e-01],
       [-9.99352336e-01, -9.99339044e-01, -9.99306679e-01, ...,
        -9.99599993e-01, -9.99699354e-01, -9.99706149e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

'mfcc'

(12565, 20, 87)

...

(12565, 84, 87)

array([[-2.13950068e-01, -2.38894999e-01, -3.80664200e-01, ...,
        -1.41635209e-01, -1.01275146e-02, -1.78813934e-07],
       [-4.45840538e-01, -5.22175133e-01, -6.68615699e-01, ...,
        -4.22790468e-01, -2.85212219e-01, -2.24623069e-01],
       [-7.63046622e-01, -7.80026078e-01, -9.83664155e-01, ...,
        -7.36702800e-01, -5.25470257e-01, -5.25247097e-01],
       ...,
       [-2.00844555e+01, -7.19054260e+01, -6.65617599e+01, ...,
        -4.13212929e+01, -4.68384933e+01, -2.91117020e+01],
       [-2.03201313e+01, -6.45224457e+01, -6.51016922e+01, ...,
        -4.41159325e+01, -4.89345970e+01, -3.10661354e+01],
       [-2.05067978e+01, -6.17005310e+01, -5.89219780e+01, ...,
        -5.76679039e+01, -5.61718521e+01, -3.33402481e+01]])

'id'

(12565,)

'XC483906.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 128, 87)

array([[-1.19495122e-02,  0.00000000e+00, -5.92639437e-03, ...,
        -9.61586274e-03, -1.45224054e-04, -1.35156866e-02],
       [-1.32104889e-01, -1.48279309e-01, -2.04118401e-01, ...,
        -2.71819055e-01, -2.12780029e-01, -1.90217406e-01],
       [-1.77966341e-01, -1.93037197e-01, -2.48268843e-01, ...,
        -3.41875643e-01, -2.82413810e-01, -2.60217577e-01],
       ...,
       [-8.56934249e-01, -8.21815670e-01, -8.13137054e-01, ...,
        -9.90914822e-01, -9.94373798e-01, -9.93971646e-01],
       [-9.33930933e-01, -9.18021381e-01, -9.22511458e-01, ...,
        -9.97301877e-01, -9.96644974e-01, -9.95461941e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -9.97628629e-01, -9.96140599e-01]])

'mfcc'

(3318, 20, 87)

...

(3318, 84, 87)

array([[ -0.81876791,  -0.83223093,  -0.95862055, ...,  -1.04782999,
         -0.91178417,  -0.89874911],
       [ -1.06328416,  -1.15007341,  -1.30937183, ...,  -1.50782466,
         -1.36429203,  -1.29628861],
       [ -1.31883109,  -1.31582081,  -1.49792707, ...,  -2.05177641,
         -1.80057609,  -1.77171004],
       ...,
       [-18.08393478, -52.00915909, -46.19884491, ..., -56.79980087,
        -66.42909241, -28.33953667],
       [-17.75449371, -48.76427078, -46.56741714, ..., -68.17790222,
        -62.38970947, -32.60255051],
       [-17.46328926, -54.79916   , -53.8106575 , ..., -59.53269958,
        -60.99742126, -34.56028748]])

'id'

(3318,)

'XC428404.mp3'

# **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 21228)

In [18]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 21228)

Fit the model with training data

In [19]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [20]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5569620253164557


In [21]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.58      0.49      0.54       455
           1       0.33      0.70      0.45       492
           2       0.80      0.53      0.64       889
           3       0.82      0.86      0.84       150
           4       0.10      0.01      0.03        67
           5       0.33      0.19      0.24        43
           6       0.56      0.38      0.45        24
           7       0.96      0.98      0.97        44
           8       0.70      0.66      0.68        50
           9       0.80      0.77      0.79       169
          10       0.80      0.66      0.72        53
          11       0.81      0.33      0.47        66
          12       0.66      0.75      0.70        59
          13       0.46      0.56      0.51        57
          14       0.38      0.74      0.50        38
          15       0.29      0.18      0.22       311
          16       0.96      0.96      0.96   

### **With MFCCs only**

In [22]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 1740)

In [23]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 1740)

In [24]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5831826401446655


In [26]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.66      0.82      0.73       455
           1       0.35      0.65      0.45       492
           2       0.75      0.40      0.52       889
           3       0.88      0.85      0.86       150
           4       0.12      0.04      0.07        67
           5       0.00      0.00      0.00        43
           6       0.16      0.17      0.16        24
           7       0.59      0.95      0.73        44
           8       0.60      0.74      0.66        50
           9       0.92      0.90      0.91       169
          10       0.73      0.42      0.53        53
          11       0.78      0.27      0.40        66
          12       0.66      0.71      0.68        59
          13       0.45      0.65      0.53        57
          14       0.40      0.71      0.51        38
          15       0.50      0.40      0.45       311
          16       0.93      0.98      0.96   

### **With Mel-Spectrogram Only**

In [27]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 11136)

In [28]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 11136)

In [29]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [30]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5424954792043399


In [31]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.60      0.60      0.60       455
           1       0.33      0.66      0.44       492
           2       0.71      0.44      0.54       889
           3       0.87      0.89      0.88       150
           4       0.08      0.01      0.03        67
           5       0.07      0.07      0.07        43
           6       0.50      0.33      0.40        24
           7       0.95      0.89      0.92        44
           8       0.65      0.52      0.58        50
           9       0.90      0.67      0.77       169
          10       0.73      0.70      0.71        53
          11       0.58      0.27      0.37        66
          12       0.70      0.68      0.69        59
          13       0.52      0.61      0.56        57
          14       0.34      0.71      0.46        38
          15       0.43      0.32      0.37       311
          16       0.85      1.00      0.92   

## **With Chroma Only**

In [32]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 1044)

In [33]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 1044)

In [34]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [35]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4933694996986136


In [36]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.47      0.52      0.50       455
           1       0.27      0.48      0.35       492
           2       0.54      0.55      0.54       889
           3       0.56      0.42      0.48       150
           4       0.00      0.00      0.00        67
           5       0.40      0.05      0.08        43
           6       0.25      0.08      0.12        24
           7       0.97      0.82      0.89        44
           8       0.43      0.24      0.31        50
           9       0.80      0.80      0.80       169
          10       0.89      0.45      0.60        53
          11       0.37      0.11      0.16        66
          12       0.80      0.73      0.76        59
          13       0.42      0.53      0.47        57
          14       0.74      0.74      0.74        38
          15       0.51      0.47      0.49       311
          16       0.72      0.51      0.60   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **With CQT Only**

In [37]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 7308)

In [38]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 7308)

In [39]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [40]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.41199517781796263


In [41]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.45      0.44      0.44       455
           1       0.29      0.76      0.41       492
           2       0.70      0.40      0.51       889
           3       0.60      0.35      0.44       150
           4       0.00      0.00      0.00        67
           5       0.07      0.02      0.03        43
           6       0.43      0.38      0.40        24
           7       0.94      1.00      0.97        44
           8       0.33      0.54      0.41        50
           9       0.41      0.21      0.28       169
          10       0.56      0.51      0.53        53
          11       0.78      0.32      0.45        66
          12       0.52      0.47      0.50        59
          13       0.33      0.47      0.39        57
          14       0.40      0.37      0.38        38
          15       0.08      0.06      0.07       311
          16       0.75      0.89      0.82   

## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 12876)

In [43]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 12876)

In [44]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5524412296564195


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.62      0.62      0.62       455
           1       0.34      0.65      0.44       492
           2       0.74      0.46      0.57       889
           3       0.87      0.87      0.87       150
           4       0.21      0.04      0.07        67
           5       0.00      0.00      0.00        43
           6       0.33      0.29      0.31        24
           7       0.93      0.91      0.92        44
           8       0.60      0.58      0.59        50
           9       0.87      0.76      0.81       169
          10       0.67      0.72      0.69        53
          11       0.68      0.26      0.37        66
          12       0.69      0.73      0.71        59
          13       0.49      0.56      0.52        57
          14       0.29      0.68      0.41        38
          15       0.36      0.27      0.31       311
          16       0.89      1.00      0.94   

## **With MFCCs and Chroma**

In [47]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 2784)

In [48]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 2784)

In [49]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [50]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6130198915009042


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.65      0.80      0.72       455
           1       0.37      0.63      0.46       492
           2       0.79      0.50      0.61       889
           3       0.84      0.85      0.84       150
           4       0.13      0.04      0.07        67
           5       0.00      0.00      0.00        43
           6       0.58      0.29      0.39        24
           7       0.71      0.95      0.82        44
           8       0.64      0.70      0.67        50
           9       0.95      0.95      0.95       169
          10       0.89      0.64      0.75        53
          11       0.86      0.36      0.51        66
          12       0.65      0.73      0.69        59
          13       0.46      0.63      0.53        57
          14       0.38      0.84      0.52        38
          15       0.51      0.41      0.46       311
          16       0.95      0.95      0.95   

## **With MFCCs and CQT**

In [52]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 9048)

In [53]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 9048)

In [54]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [55]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5325497287522604


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.58      0.49      0.53       455
           1       0.30      0.75      0.43       492
           2       0.84      0.45      0.58       889
           3       0.86      0.78      0.82       150
           4       0.06      0.01      0.02        67
           5       0.00      0.00      0.00        43
           6       0.35      0.38      0.36        24
           7       0.90      1.00      0.95        44
           8       0.56      0.66      0.61        50
           9       0.90      0.84      0.87       169
          10       0.65      0.53      0.58        53
          11       0.71      0.26      0.38        66
          12       0.69      0.75      0.72        59
          13       0.42      0.53      0.47        57
          14       0.41      0.66      0.51        38
          15       0.32      0.18      0.23       311
          16       0.82      0.98      0.89   

## **With Chroma and CQT**

In [57]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 8352)

In [58]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 8352)

In [59]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [60]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4349005424954792


In [61]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.44      0.43      0.43       455
           1       0.28      0.77      0.41       492
           2       0.70      0.40      0.51       889
           3       0.63      0.46      0.53       150
           4       0.00      0.00      0.00        67
           5       0.18      0.05      0.07        43
           6       0.56      0.42      0.48        24
           7       0.92      1.00      0.96        44
           8       0.57      0.62      0.60        50
           9       0.52      0.31      0.39       169
          10       0.73      0.70      0.71        53
          11       0.89      0.36      0.52        66
          12       0.50      0.49      0.50        59
          13       0.39      0.47      0.43        57
          14       0.53      0.76      0.62        38
          15       0.08      0.04      0.06       311
          16       0.86      0.91      0.88   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.8122066581974924
F1 Score (Weighted): 0.4268732780638383


## **With Mel-Spectrogram and Chroma**

In [62]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 12180)

In [63]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 12180)

In [64]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5458107293550332


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.61      0.62      0.62       455
           1       0.34      0.70      0.46       492
           2       0.71      0.45      0.55       889
           3       0.81      0.89      0.85       150
           4       0.00      0.00      0.00        67
           5       0.05      0.05      0.05        43
           6       0.29      0.29      0.29        24
           7       0.95      0.89      0.92        44
           8       0.59      0.58      0.59        50
           9       0.77      0.70      0.74       169
          10       0.77      0.70      0.73        53
          11       0.67      0.24      0.36        66
          12       0.71      0.78      0.74        59
          13       0.58      0.60      0.59        57
          14       0.40      0.71      0.51        38
          15       0.42      0.27      0.33       311
          16       0.87      1.00      0.93   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 18444)

In [68]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 18444)

In [69]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [70]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5394816154309825


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.51      0.51       455
           1       0.34      0.70      0.46       492
           2       0.76      0.52      0.62       889
           3       0.80      0.85      0.82       150
           4       0.07      0.01      0.02        67
           5       0.17      0.16      0.17        43
           6       0.38      0.38      0.38        24
           7       0.93      0.95      0.94        44
           8       0.59      0.48      0.53        50
           9       0.83      0.70      0.76       169
          10       0.67      0.62      0.65        53
          11       0.57      0.24      0.34        66
          12       0.67      0.69      0.68        59
          13       0.57      0.61      0.59        57
          14       0.51      0.71      0.59        38
          15       0.33      0.20      0.25       311
          16       0.89      0.98      0.93   

## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 20184)

In [73]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 20184)

In [74]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [75]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5533453887884268


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.57      0.49      0.53       455
           1       0.33      0.68      0.44       492
           2       0.79      0.55      0.65       889
           3       0.86      0.85      0.86       150
           4       0.12      0.01      0.03        67
           5       0.15      0.05      0.07        43
           6       0.45      0.42      0.43        24
           7       0.93      0.95      0.94        44
           8       0.65      0.60      0.62        50
           9       0.83      0.79      0.81       169
          10       0.75      0.68      0.71        53
          11       0.68      0.23      0.34        66
          12       0.69      0.75      0.72        59
          13       0.46      0.58      0.52        57
          14       0.36      0.71      0.48        38
          15       0.28      0.17      0.22       311
          16       0.93      0.98      0.96   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 13920)

In [78]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 13920)

In [79]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [80]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5650994575045208


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.65      0.62      0.63       455
           1       0.34      0.66      0.45       492
           2       0.73      0.48      0.58       889
           3       0.88      0.85      0.87       150
           4       0.25      0.03      0.05        67
           5       0.11      0.05      0.07        43
           6       0.43      0.38      0.40        24
           7       0.97      0.89      0.93        44
           8       0.68      0.56      0.62        50
           9       0.81      0.78      0.80       169
          10       0.76      0.70      0.73        53
          11       0.80      0.30      0.44        66
          12       0.64      0.71      0.67        59
          13       0.52      0.67      0.58        57
          14       0.38      0.79      0.51        38
          15       0.39      0.30      0.34       311
          16       0.85      1.00      0.92   

## Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.557
3,chroma,1.0,0.493
8,chroma_cqt,1.0,0.435
4,cqt,1.0,0.412
2,melspectrogram,1.0,0.542
9,melspectrogram_chroma,1.0,0.546
12,melspectrogram_chroma_mfcc,1.0,0.565
10,melspectrogram_cqt,1.0,0.539
11,melspectrogram_cqt_mfcc,1.0,0.553
1,mfcc,1.0,0.583


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.56       0.88
3                    chroma      0.49       0.85
8                chroma_cqt      0.43       0.81
4                       cqt      0.40       0.80
2            melspectrogram      0.54       0.89
9     melspectrogram_chroma      0.54       0.89
10       melspectrogram_cqt      0.54       0.87
11  melspectrogram_cqt_mfcc      0.57       0.90
1                      mfcc      0.58       0.91
6               mfcc_chroma      0.61       0.91
7                  mfcc_cqt      0.54       0.89
5       mfcc_melspectrogram      0.55       0.90
